# Q-tabular learning
- Q-tabular learning don't need to predict all possible states, it just use the history to predict the states that will happen when comparing to Q-learning.
1. start from Q(s,a) empty table
2. get (s, a, r, s') from environment
3. use Bellman update Q(s,a) = (1-ALPHA)Q(s,a) + ALPHA(reward + GAMMA * max_action_a'Q(s',a'))
4. Check for convergence, if not, repeat step 2.

In [1]:
import gym
import collections
from tensorboardX import SummaryWriter

In [2]:
ENV_NAME = "FrozenLake-v1"
GAMMA = 0.9
#learning rate
ALPHA = 0.2
TEST_EPISODES = 20

In [3]:
#we don't need to remover reward and transfer table, just value table is enough, so memory used is much less
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)
    
    #we get the transition of environment
    def sample_env(self):
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return (old_state, action, reward, new_state)
    
    #get the biggest value on the state for action, this function will run twice, first for policy evaluation and 
    # second for updating value
    def best_value_and_action(self, state):
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action
    
    #we calculate the new value with Bellman equation and get the previous state action value,
    # then we mix the old value with the new value with the learning rate ALPHA
    def value_update(self, s, a, r, next_s):
        best_v, _ = self.best_value_and_action(next_s)
        new_val = r + GAMMA * best_v
        old_val = self.values[(s, a)]
        self.values[(s, a)] = old_val * (1-ALPHA) + new_val * ALPHA
    
    #agent will use the environment given to play one episode, each step will use the q tabular value to check our policy.
    #this won't change the value table, just use the value table to check the best action
    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

In [4]:
# run a step and update value, play and test the value to know the reward on policy, if reward is good enough,
# we stop training
if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-q-learning")

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        s, a, r, next_s = agent.sample_env()
        agent.value_update(s, a, r, next_s)

        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TEST_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()

Best reward updated 0.000 -> 0.100
Best reward updated 0.100 -> 0.150
Best reward updated 0.150 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.450
Best reward updated 0.450 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 6958 iterations!


# we need more iteration than value iteration because we won't use the test policy experience for updating q-values